In [6]:
import torch
import numpy as np

def convolution_transpose(matrix, in_channels, out_channels, kernel_size,
                          stride=1, padding=0, output_padding=0, dilation=1,
                          bias=True, padding_mode='zeros'):
    # Генерация случайных значений для смещения (bias) или заполнение нулями
    bias_val = torch.rand(out_channels) if bias else torch.zeros(out_channels)

    # Генерация случайных значений для весов или создание с указанными размерами
    weights = torch.rand(in_channels, out_channels, kernel_size, kernel_size) if isinstance(kernel_size, int) else torch.rand(in_channels, out_channels, *kernel_size)

    # Список для хранения результатов тензоров
    res_tensor = []

    # Итерация по выходным каналам
    for l in range(out_channels):
        # Инициализация feature_map нулями
        feature_map = torch.zeros((matrix.shape[1] - 1) * stride + dilation * (kernel_size - 1) + 1,
                                  (matrix.shape[2] - 1) * stride + dilation * (kernel_size - 1) + 1)

        # Итерация по входным каналам
        for c in range(in_channels):
            for i in range(0, matrix.shape[1]):
                for j in range(0, matrix.shape[2]):
                    # Получение значения из входной матрицы и соответствующего веса
                    val = matrix[c][i][j]
                    proizv = val * weights[c][l]

                    # Инициализация zero_tensor для текущего значения
                    zero_tensor = torch.zeros((weights.shape[2] - 1) * dilation + 1,
                                              (weights.shape[3] - 1) * dilation + 1)

                    # Итерация для установки значений в zero_tensor
                    for a in range(0, zero_tensor.shape[0], dilation):
                        for b in range(0, zero_tensor.shape[1], dilation):
                            zero_tensor[a][b] = proizv[a // dilation][b // dilation]

                    # Добавление zero_tensor к соответствующей области feature_map
                    res = np.add(zero_tensor,
                                 feature_map[i * stride:i * stride + (weights.shape[2] - 1) * dilation + 1,
                                 j * stride:j * stride + (weights.shape[3] - 1) * dilation + 1])
                    feature_map[i * stride:i * stride + (weights.shape[2] - 1) * dilation + 1,
                    j * stride:j * stride + (weights.shape[3] - 1) * dilation + 1] = res

        # Добавление смещения (bias) и обрезка feature_map
        res_tensor.append(np.add(feature_map, np.full(feature_map.shape, bias_val[l])))

    # Применение output_padding и обрезка для каждого тензора в результате
    for t in range(len(res_tensor)):
        if output_padding > 0:
            pad_func = torch.nn.ConstantPad1d((0, output_padding, 0, output_padding), 0)
            res_tensor[t] = pad_func(res_tensor[t])

        res_tensor[t] = res_tensor[t][0 + padding:res_tensor[t].shape[0] - padding,
                                      0 + padding:res_tensor[t].shape[1] - padding]

    return res_tensor, weights, torch.tensor(bias_val)


In [7]:
# Создаем тестовый тензор
import torch
import numpy as np

# Создаем входной тензор
input_tensor = torch.rand(3, 4, 4)

# Параметры для ConvTranspose
in_channels = 3
out_channels = 2
kernel_size = 3
stride = 2
padding = 1
output_padding = 1
dilation = 1
bias = True
padding_mode = 'zeros'

# Вызываем custom_conv_transpose
output_result, weights, bias_values = custom_conv_transpose(input_tensor, in_channels, out_channels, kernel_size,
                                                             stride, padding, output_padding, dilation,
                                                             bias, padding_mode)

# Выводим результаты
print("Output Tensor:")
for tensor in output_result:
    print(tensor)

print("\nWeights:")
print(weights)

print("\nBias Values:")
print(bias_values)


Output Tensor:
tensor([[0.9027, 1.9098, 0.9352, 2.5715, 1.1801, 2.9071, 0.9434, 1.4642],
        [2.0210, 3.6973, 1.8302, 4.5274, 3.9776, 5.1546, 2.7033, 2.3064],
        [1.0215, 1.9005, 0.6667, 2.0264, 0.8654, 2.3572, 1.0171, 1.7830],
        [2.7187, 3.6481, 2.0825, 4.4231, 3.3661, 5.2843, 3.6773, 3.0200],
        [0.7599, 1.9878, 0.7391, 2.3780, 0.9574, 2.9887, 1.2266, 2.0940],
        [2.8173, 4.4106, 3.0571, 4.5421, 3.1799, 4.6192, 3.3037, 2.2478],
        [1.0212, 2.6195, 0.9294, 2.3941, 0.7755, 1.9281, 0.8717, 1.2733],
        [1.6829, 2.8950, 1.9624, 2.3634, 1.3139, 1.8695, 1.3328, 0.8618]])
tensor([[1.6182, 1.9939, 1.6253, 2.2668, 2.5872, 2.8660, 1.8212, 1.3270],
        [2.1402, 2.2587, 1.4978, 3.3080, 2.9704, 3.7707, 2.2100, 1.9179],
        [1.8631, 1.7294, 1.2127, 1.9745, 1.9138, 2.2344, 2.1790, 1.9547],
        [2.0346, 2.9921, 1.6645, 3.1610, 2.4412, 3.4814, 3.3167, 2.5313],
        [1.6997, 2.0906, 1.5906, 2.3979, 2.1951, 3.0739, 2.5839, 1.8983],
        [2.6819, 3.527

C:\Users\pospi\AppData\Local\Temp\ipykernel_12772\3736592967.py:66: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return res_tensor, weights, torch.tensor(bias_val)


In [8]:
import torch
import unittest

class TestConvolutionTranspose(unittest.TestCase):

    def test_convolution_transpose_no_bias(self):
        input_matrix = torch.randn(2, 4, 4)  # 2 channels, 4x4 matrix
        in_channels = 2
        out_channels = 3
        kernel_size = 2
        stride = 2
        padding = 1
        output_padding = 0
        dilation = 1
        bias = False
        padding_mode = 'zeros'

        my_res, kernel, bias_val = custom_conv_transpose(
            input_matrix,
            in_channels=in_channels, out_channels=out_channels,
            kernel_size=kernel_size, stride=stride,
            padding=padding, output_padding=output_padding,
            dilation=dilation, bias=bias,
            padding_mode=padding_mode,
        )

        # Check if the PyTorch convolution transpose layer has bias
        if bias:
            # If bias is True, use the provided bias value
            torch_function = torch.nn.ConvTranspose2d(
                in_channels=in_channels, out_channels=out_channels,
                kernel_size=kernel_size, stride=stride,
                padding=padding, output_padding=output_padding,
                dilation=dilation, bias=bias,
                padding_mode=padding_mode,
            )
            torch_function.weight.data = kernel
            torch_function.bias.data = bias_val
        else:
            # If bias is False, create a ConvTranspose2d layer without bias
            torch_function = torch.nn.ConvTranspose2d(
                in_channels=in_channels, out_channels=out_channels,
                kernel_size=kernel_size, stride=stride,
                padding=padding, output_padding=output_padding,
                dilation=dilation, bias=False,  # Set bias to False
                padding_mode=padding_mode,
            )
            torch_function.weight.data = kernel

        result = str(np.round([tensor.tolist() for tensor in my_res], 2))
        torch_res = str(np.round(torch_function(input_matrix).data.numpy(), 2))

        # Corrected indentation for the assertion
        self.assertEqual(result, torch_res)

In [9]:
unittest.main(argv=[''], exit=False)

C:\Users\pospi\AppData\Local\Temp\ipykernel_12772\3736592967.py:66: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return res_tensor, weights, torch.tensor(bias_val)
.
----------------------------------------------------------------------
Ran 1 test in 0.016s

OK
